#Ingestão Base 2

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

file_path = "/content/Base 2 - Vendas Clientes ConsigCar.xlsx"


df = pd.read_excel(file_path)

# Remove 'R$', pontos e converte a vírgula decimal para ponto
df['Valor parcela'] = df['Valor parcela'].replace({'R\$': '', '\.': '', ',': '.'}, regex=True).astype(float)

# Converte a coluna de data e remove a hora
df['Data do Pagamento'] = pd.to_datetime(df['Data do Pagamento']).dt.date

# Cria conexão com banco de dados SQLite
engine = create_engine("sqlite:///vendas_clientes_consigcar.db")

# Insere a tabela no banco
df.to_sql("vendas_clientes_consigcar", con=engine, if_exists="replace", index=False)



207

In [ ]:
df.head()

,Data do Pagamento,Tipo Produto,Nome,WhatsApp,Quantidade de vezes,Valor parcela,Vendedor,Comentarios
0,2024-11-11,Completa,Fernanda Gomes Ibrahim Campos,(11) 99123-4567,18,1834.66,Thiago,NaN
1,2024-11-22,Completa,Patrícia Zanetti Jardim Campos Souza,(21) 99234-5678,6,641.33,Thiago,NaN
2,2024-11-18,Completa,Eduardo Yamada Dias Vieira Nogueira,(43) 99345-6789,18,1879.30,Thiago,NaN
3,2024-10-22,Completa,Thiago Lopes Xavier Queiroz,(51) 99456-7890,18,1831.83,Thiago,NaN
4,2024-10-20,Completa,Vanessa Yamada Farias Krause Teixeira,(61) 99567-8901,6,645.73,Thiago,NaN


,Nome (Alucar),Data,Mes,Ano,Valor\nReceita,Unnamed: 5,Faturamento\nConsigCar,Valor
0,JANEIRO,NaN,NaN,NaN,NaN,NaN,JANEIRO,NaN
1,João Carlos Silva Pereira,2025-01-02 00:00:00,1.0,2025.0,1432.0,NaN,Pagseguro,35918.27
2,Maria Fernanda Souza Oliveira,2025-01-03 00:00:00,1.0,2025.0,4189.0,NaN,NaN,NaN
3,Marcos Vinícius Silva Costa,2025-01-06 00:00:00,1.0,2025.0,118.0,NaN,NaN,NaN
4,Ana Beatriz Almeida Lima,2025-01-07 00:00:00,1.0,2025.0,2756.0,NaN,NaN,NaN


#Ingestão Base 1

In [ ]:
#função para editar a tabela de Receita
def edit_sheet(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df.columns = df.columns.str.replace('\n', '_')
    df.columns = df.columns.str.replace(' ', '_')

    return df

In [ ]:
import requests
import pandas as pd
from io import BytesIO

#Definindo ID da planilha e url
sheet_id = '1cucnW4yVosO5n5BFgwXYv6rVy8yj6NTasM83RTCMOug'
sheet_url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=xlsx'

#Salvando arquivo em FILE
response = requests.get(sheet_url)
file = BytesIO(response.content)

#Lendo e aramazenando em um dataframe as tabelas
df_Receita = pd.read_excel(file, sheet_name='Receita', index_col=False)
df_Receita = edit_sheet(df_Receita)
df_Despesas = pd.read_excel(file, sheet_name='Despesas', index_col=False)

In [ ]:
#Inciano tratamento
#exclusão de linhas sem dados nas colunas Data e Mês
df_Receita = df_Receita.dropna(subset=["Data", "Mes"])

#Transformação da cluna data
df_Receita['Data'] = pd.to_datetime(df_Receita['Data']).dt.date

#transformação das colunas Mês e Ano para inteiro
df_Receita["Mes"] = df_Receita["Mes"].astype(int)
df_Receita["Ano"] = df_Receita["Ano"].astype(int)

#colunas = df_Receita.columns

#Separação das colunas da Alucar e Consigcar
df_Receita_Alucar = df_Receita.iloc[:, 0:5] \
                    .reset_index(drop = True)

df_Receita_ConsigCar = df_Receita.iloc[:, [1,6]] \
                        .dropna(subset=["Valor"]) \
                        .reset_index(drop = True)

#Separação do dataframe da Receita da Alucar e estimativa da Alucar
df_Receita_Alucar_estimativa = df_Receita_Alucar[df_Receita_Alucar["Nome_(Alucar)"] == 'Estimativa'] \
                                .reset_index(drop = True)

df_Receita_Alucar = df_Receita_Alucar[df_Receita_Alucar["Nome_(Alucar)"] != 'Estimativa']






<ipython-input-5-add6acc71da5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Receita['Data'] = pd.to_datetime(df_Receita['Data']).dt.date
<ipython-input-5-add6acc71da5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Receita["Mes"] = df_Receita["Mes"].astype(int)
<ipython-input-5-add6acc71da5>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [ ]:
df_Receita_ConsigCar.head()

,Data,Valor
0,2025-01-02,35918.27
1,2025-02-01,51472.94
2,2025-03-01,34994.22
3,2025-04-01,48182.77


In [ ]:
df_Receita_Alucar.tail(30)

,Nome_(Alucar),Data,Mes,Ano,Valor_Receita
51,Thiago Lima Costa Almeida,2025-03-13,3,2025,4591.0
52,Daniela Costa de Almeida,2025-03-13,3,2025,1189.0
53,Roberta Pereira Lima Santos,2025-03-18,3,2025,2721.0
54,Maurício de Souza Lima,2025-03-19,3,2025,4469.0
55,Maria Clara Pereira Lima,2025-03-20,3,2025,939.0
56,Lucas Matheus Costa Pereira,2025-03-20,3,2025,2139.0
57,Rosa Costa de Lima Pereira,2025-03-21,3,2025,3629.0
58,Paulo Ricardo Almeida Lima,2025-03-29,3,2025,489.0
59,Sofia Gomes da Silva,2025-03-31,3,2025,1581.0
60,Eduardo Lima Souza Costa,2025-03-31,3,2025,4221.0


In [ ]:
df_Receita_Alucar_estimativa.head()

,Nome_(Alucar),Data,Mes,Ano,Valor_Receita
0,Estimativa,2025-06-01,6,2025,36000.0
1,Estimativa,2025-07-01,7,2025,36000.0
2,Estimativa,2025-08-01,8,2025,36000.0
3,Estimativa,2025-09-01,9,2025,36000.0
4,Estimativa,2025-10-01,10,2025,36000.0


#Ingestão da planilha despesas

In [ ]:
#Excluindo colunas desnecessárias
df_Despesas.columns = df_Despesas.iloc[0]
df_Despesas = df_Despesas[1:]
df_Despesas.reset_index(drop = True, inplace = True)

df_Despesas.head(30)


,DESPESAS,janeiro,fevereiro,março,abril,maio,junho,julho,agosto,setembro,outubro,novembro,dezembro
0,Lucas Pereira Costa,7325,5528,2358,6523.35,6225.78,6552,4323,4635.26,3368.22,7856.25,7752,5823
1,Mariana Oliveira Souza,1892.41,1892.41,2586,2789.12,2789.12,2789.12,2878.36,2789.12,2789.12,1892.41,2789.12,2789.12
2,João Henrique Lima,2600.22,3700.22,2265.36,2600.35,3369.22,4521,6632,3587,5832.33,3259.33,7823.33,4369.22
3,Ana Paula Santos,1756.23,1756.23,1756.23,2456.78,2456.78,2456.78,1756.23,2456.78,2456.78,1756.23,2456.78,2456.78
4,Aluguel de escritório,1800,1800,1800,1800,1800,1800,1800,1800,1800,1800,1800,1800
5,Rafael Martins Silva,1421.67,1421.67,1421.67,2987.65,2987.65,2987.65,2992.35,2987.65,2987.65,2997,2987.65,2987.65
6,Contas de energia elétrica,2089.34,2089.34,2089.34,1654.32,1654.32,1654.32,2089.34,1654.32,1654.32,2089.34,1654.32,1654.32
7,Contas de água e esgoto,225.25,452,318.92,278.98,235,275,325,222,412,318.92,321.96,278.98
8,Impostos e taxas,1956.11,1956.11,1956.11,3121.45,3121.45,3121.45,1956.11,3121.45,3121.45,1956.11,3121.45,3121.45
9,Despesas com matéria-prima,628.75,628.75,628.75,2345.67,2345.67,2345.67,628.75,2345.67,2345.67,628.75,2345.67,2345.67


In [ ]:
#Separando Despesas da Alucar e ConsigCar

# Encontrar o índice da linha que contém TOTAL na coluna Despesa
index_total = df_Despesas[df_Despesas['DESPESAS'].str.contains("TOTAL", na=False)].index.min()

# Selecionar todas as linhas até a linha anterior à que contém TOTAL
df_Despesas_Alucar = df_Despesas.iloc[:index_total].copy()
df_Despesas_Alucar = df_Despesas_Alucar.reset_index(drop = True) \
                                       .melt(id_vars=["DESPESAS"], var_name="Mês", value_name="Valor")

# Selecionar as linhas referentes a ConsigCar
df_Despesas_ConsigCar = df_Despesas.iloc[index_total+4:-1] \
                        .copy().reset_index(drop = True) \
                        .melt(id_vars=["DESPESAS"], var_name="Mês", value_name="Valor")


In [ ]:
df_Despesas_ConsigCar.head(30)

,DESPESAS,Mês,Valor
0,Ricardo Augusto da Silva Pereira,janeiro,3482.76
1,Juliana Costa Almeida Lima,janeiro,6215.43
2,Gabriel Henrique Oliveira Souza,janeiro,4823.36
3,Larissa Beatriz Santos Ribeiro,janeiro,4890.22
4,Felipe Augusto Souza Lima,janeiro,2658.9
5,Mariana Oliveira Costa Silva,janeiro,9732.25
6,Carlos Eduardo Pereira Lima,janeiro,1847.35
7,Trafego,janeiro,3482.76
8,Google Ads,janeiro,6215.43
9,Meta Ads,janeiro,1973.5


In [ ]:
df_Despesas_Alucar.head(35)

,DESPESAS,Mês,Valor
0,Lucas Pereira Costa,janeiro,7325
1,Mariana Oliveira Souza,janeiro,1892.41
2,João Henrique Lima,janeiro,2600.22
3,Ana Paula Santos,janeiro,1756.23
4,Aluguel de escritório,janeiro,1800
5,Rafael Martins Silva,janeiro,1421.67
6,Contas de energia elétrica,janeiro,2089.34
7,Contas de água e esgoto,janeiro,225.25
8,Impostos e taxas,janeiro,1956.11
9,Despesas com matéria-prima,janeiro,628.75
